In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import random
tf.random.set_seed(3407)
np.random.seed(3407)
random.seed(3407)

from helper_functions import (
    get_file_size, 
    convert_bytes, 
    convert_prefetchdataset_to_numpy_arrays,
    predict_and_print_full_results,
    evaluate_time_of_prediction
    )

2024-06-18 22:09:47.514618: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-18 22:09:47.543676: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-18 22:09:47.543698: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-18 22:09:47.544423: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-18 22:09:47.548999: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-18 22:09:47.549669: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [3]:
sample_rate = 16000

In [4]:
train_dataset = tf.keras.utils.audio_dataset_from_directory("dataset/training", labels='inferred', sampling_rate=16000, batch_size=32, shuffle=True, seed=3407)
test_dataset = tf.keras.utils.audio_dataset_from_directory("dataset/testing", labels='inferred', sampling_rate=16000, batch_size=32, shuffle=True, seed=3407)
val_dataset = tf.keras.utils.audio_dataset_from_directory("dataset/validation", labels='inferred', sampling_rate=16000, batch_size=32, shuffle=True, seed=3407)

label_names = np.array(test_dataset.class_names)
print("Classes: ", label_names)

Found 11292 files belonging to 2 classes.


2024-06-18 22:10:06.470252: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-18 22:10:06.470559: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-06-18 22:10:06.547894: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
2024-06-18 22:10:06.565007: W tensorflow_io/core/kernels/audio_video_mp3_kernels.cc:271] libmp3lame.so.0 or lame functions are not available


Found 1393 files belonging to 2 classes.
Found 1380 files belonging to 2 classes.
Classes:  ['non_target' 'target']


In [5]:
x_train_np, y_train_np = convert_prefetchdataset_to_numpy_arrays(train_dataset, data_type="time-series")
x_val_np, y_val_np = convert_prefetchdataset_to_numpy_arrays(val_dataset, data_type="time-series")
x_test_np, y_test_np = convert_prefetchdataset_to_numpy_arrays(test_dataset, data_type="time-series")

In [6]:
BASE_MODEL_FILE_NAME = "time_series_models_from_notebooks/cnn/cnn_time_series_16kHz_baseline.keras"
baseline_model = tf.keras.models.load_model(BASE_MODEL_FILE_NAME)
baseline_model.summary()
convert_bytes(get_file_size(BASE_MODEL_FILE_NAME), "MB")

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 47998, 6)          24        
                                                                 
 max_pooling1d (MaxPooling1  (None, 5999, 6)           0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 5997, 8)           152       
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 599, 8)            0         
 g1D)                                                            
                                                                 
 reshape (Reshape)           (None, 4792)              0         
                                                                 
 dense (Dense)               (None, 12)                5

### Convert to TF Lite format

In [7]:
converter = tf.lite.TFLiteConverter.from_keras_model(baseline_model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpojacw4sv/assets


INFO:tensorflow:Assets written to: /tmp/tmpojacw4sv/assets
2024-06-18 22:10:47.467914: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-06-18 22:10:47.467958: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-06-18 22:10:47.468261: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpojacw4sv
2024-06-18 22:10:47.469356: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-06-18 22:10:47.469368: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpojacw4sv
2024-06-18 22:10:47.472287: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-06-18 22:10:47.473129: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-06-18 22:10:47.509152: I tensorflow/cc/saved_model/loader.cc:217] Running initializatio

In [8]:
print("Validation dataset:")
(
    y_pred_val, 
    non_overlap_patritions_f1_scores_val, 
    bootstrap_patritions_f1_scores_val,
    ) = predict_and_print_full_results(tflite_model, x_val_np, y_val_np, model_format="tf_lite")

print("\nTest dataset:")
(
    y_pred_test, 
    non_overlap_patritions_f1_scores_test, 
    bootstrap_patritions_f1_scores_test,
    ) = predict_and_print_full_results(tflite_model, x_test_np, y_test_np, model_format="tf_lite")

print("\nTime for Test dataset:")
evaluate_time_of_prediction(tflite_model, x_test_np, y_test_np, model_format="tf_lite")

TF_LITE_MODEL_FILE_NAME = "time_series_models_from_notebooks/cnn/cnn_time_series_16kHz.tflite"
open(TF_LITE_MODEL_FILE_NAME, "wb").write(tflite_model)
print("\n")
print("Model file name: ", TF_LITE_MODEL_FILE_NAME)
convert_bytes(get_file_size(TF_LITE_MODEL_FILE_NAME), "MB")

Validation dataset:


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Basic assessment of the whole dataset (without any partitions):
Accuracy: 95.07%
Recall: 88.35%
Precision: 96.98%
F1-score: 92.46%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9229463464959073

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9275873181529032

Test dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 96.12%
Recall: 92.14%
Precision: 95.91%
F1-score: 93.99%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9384192313336935

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9404208800514493

Time for Test dataset:
Accuracy: 96.12%
Recall: 92.14%
Precision: 95.91%
F1-score: 93.99%

Time to make a prediction for a single data point
Mean: 0.003 seconds
Max: 0.01 seconds
Min: 0.002 seconds




In [9]:
convert_bytes(get_file_size(TF_LITE_MODEL_FILE_NAME), "KB")

File size: 231.473 Kilobytes


### Dynamic range quantization

In [10]:
dynamic_range_quant_converter = tf.lite.TFLiteConverter.from_keras_model(baseline_model)
dynamic_range_quant_converter.optimizations = [tf.lite.Optimize.DEFAULT]
dynamic_range_quant_model = dynamic_range_quant_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpoxudy72i/assets


INFO:tensorflow:Assets written to: /tmp/tmpoxudy72i/assets
2024-06-18 22:12:22.094424: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-06-18 22:12:22.094466: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-06-18 22:12:22.094606: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpoxudy72i
2024-06-18 22:12:22.095499: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-06-18 22:12:22.095510: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpoxudy72i
2024-06-18 22:12:22.097855: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-06-18 22:12:22.130216: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmpoxudy72i
2024-06-18 22:12:22.141435: I tensorflow/cc/saved_model/loader.cc:316] SavedModel

In [11]:
print("Validation dataset:")
(
    y_pred_val, 
    non_overlap_patritions_f1_scores_val, 
    bootstrap_patritions_f1_scores_val,
    ) = predict_and_print_full_results(dynamic_range_quant_model, x_val_np, y_val_np, model_format="tf_lite")

print("\nTest dataset:")
(
    y_pred_test, 
    non_overlap_patritions_f1_scores_test, 
    bootstrap_patritions_f1_scores_test,
    ) = predict_and_print_full_results(dynamic_range_quant_model, x_test_np, y_test_np, model_format="tf_lite")

print("\nTime for Test dataset:")
evaluate_time_of_prediction(dynamic_range_quant_model, x_test_np, y_test_np, model_format="tf_lite")

DRQ_MODEL_FILE_NAME = "time_series_models_from_notebooks/cnn/cnn_time_series_16kHz_drq.tflite"
open(DRQ_MODEL_FILE_NAME, "wb").write(dynamic_range_quant_model)
print("\n")
print("Model file name: ", DRQ_MODEL_FILE_NAME)
convert_bytes(get_file_size(DRQ_MODEL_FILE_NAME), "MB")

Validation dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 95.07%
Recall: 88.35%
Precision: 96.98%
F1-score: 92.46%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9229463464959073

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9254718679169541

Test dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 96.12%
Recall: 92.14%
Precision: 95.91%
F1-score: 93.99%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9384192313336935

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9423030490499201

Time for Test dataset:
Accuracy: 96.12%
Recall: 92.14%
Precision: 95.91%
F1-score: 93.99%

Time to make a prediction for a single data point
Mean: 0.003 seconds
Max: 0.009 seconds


In [12]:
convert_bytes(get_file_size(DRQ_MODEL_FILE_NAME), "KB")

File size: 63.078 Kilobytes


### Float16 quantization

In [13]:
float16_quant_converter = tf.lite.TFLiteConverter.from_keras_model(baseline_model)
float16_quant_converter.optimizations = [tf.lite.Optimize.DEFAULT]
float16_quant_converter.target_spec.supported_types = [tf.float16]
float16_quant_model = float16_quant_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp0rwvba7a/assets


INFO:tensorflow:Assets written to: /tmp/tmp0rwvba7a/assets
2024-06-18 22:13:20.085479: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-06-18 22:13:20.085520: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-06-18 22:13:20.085676: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp0rwvba7a
2024-06-18 22:13:20.086791: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-06-18 22:13:20.086804: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmp0rwvba7a
2024-06-18 22:13:20.089675: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-06-18 22:13:20.125721: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmp0rwvba7a
2024-06-18 22:13:20.139014: I tensorflow/cc/saved_model/loader.cc:316] SavedModel

In [14]:
print("Validation dataset:")
(
    y_pred_val, 
    non_overlap_patritions_f1_scores_val, 
    bootstrap_patritions_f1_scores_val,
    ) = predict_and_print_full_results(float16_quant_model, x_val_np, y_val_np, model_format="tf_lite")

print("\nTest dataset:")
(
    y_pred_test, 
    non_overlap_patritions_f1_scores_test, 
    bootstrap_patritions_f1_scores_test,
    ) = predict_and_print_full_results(float16_quant_model, x_test_np, y_test_np, model_format="tf_lite")

print("\nTime for Test dataset:")
evaluate_time_of_prediction(float16_quant_model, x_test_np, y_test_np, model_format="tf_lite")

FLOAT16_QUANT_MODEL_FILE_NAME = "time_series_models_from_notebooks/cnn/cnn_time_series_16kHz_float16q.tflite"
open(FLOAT16_QUANT_MODEL_FILE_NAME, "wb").write(float16_quant_model)
print("\n")
print("Model file name: ", FLOAT16_QUANT_MODEL_FILE_NAME)
convert_bytes(get_file_size(FLOAT16_QUANT_MODEL_FILE_NAME), "MB")

Validation dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 95.07%
Recall: 88.35%
Precision: 96.98%
F1-score: 92.46%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9229463464959073

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9267552244846163

Test dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 96.12%
Recall: 92.14%
Precision: 95.91%
F1-score: 93.99%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9384192313336935

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9393093125860739

Time for Test dataset:
Accuracy: 96.12%
Recall: 92.14%
Precision: 95.91%
F1-score: 93.99%

Time to make a prediction for a single data point
Mean: 0.003 seconds
Max: 0.01 seconds
M

In [15]:
convert_bytes(get_file_size(FLOAT16_QUANT_MODEL_FILE_NAME), "KB")

File size: 119.863 Kilobytes


### Float fallback

In [18]:
float_fallback_quant_converter = tf.lite.TFLiteConverter.from_keras_model(baseline_model)
float_fallback_quant_converter.optimizations = [tf.lite.Optimize.DEFAULT]
float_fallback_quant_converter.representative_dataset = representative_data_gen
float_fallback_quant_model = float_fallback_quant_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpitb8zq9x/assets


INFO:tensorflow:Assets written to: /tmp/tmpitb8zq9x/assets
/home/polina/test/venv/lib/python3.9/site-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2024-06-18 22:15:30.681086: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-06-18 22:15:30.681127: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-06-18 22:15:30.681261: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpitb8zq9x
2024-06-18 22:15:30.682260: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-06-18 22:15:30.682270: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpitb8zq9x
2024-06-18 22:15:30.684999: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-06-18 22:15:30.

In [19]:
print("Validation dataset:")
(
    y_pred_val, 
    non_overlap_patritions_f1_scores_val, 
    bootstrap_patritions_f1_scores_val,
    ) = predict_and_print_full_results(float_fallback_quant_model, x_val_np, y_val_np, model_format="tf_lite")

print("\nTest dataset:")
(
    y_pred_test, 
    non_overlap_patritions_f1_scores_test, 
    bootstrap_patritions_f1_scores_test,
    ) = predict_and_print_full_results(float_fallback_quant_model, x_test_np, y_test_np, model_format="tf_lite")

print("\nTime for Test dataset:")
evaluate_time_of_prediction(float_fallback_quant_model, x_test_np, y_test_np, model_format="tf_lite")

FLOAT_FALLBACK_QUANT_MODEL_FILE_NAME = "time_series_models_from_notebooks/cnn/cnn_time_series_16kHz_float_fallback_q.tflite"
open(FLOAT_FALLBACK_QUANT_MODEL_FILE_NAME, "wb").write(float_fallback_quant_model)
print("\n")
print("Model file name: ", FLOAT_FALLBACK_QUANT_MODEL_FILE_NAME)
convert_bytes(get_file_size(FLOAT_FALLBACK_QUANT_MODEL_FILE_NAME), "MB")

Validation dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 94.78%
Recall: 86.65%
Precision: 97.85%
F1-score: 91.91%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.918418514295289

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.920930831367834

Test dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 95.98%
Recall: 90.61%
Precision: 96.96%
F1-score: 93.68%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9374100728942331

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9291296206954337

Time for Test dataset:
Accuracy: 95.98%
Recall: 90.61%
Precision: 96.96%
F1-score: 93.68%

Time to make a prediction for a single data point
Mean: 0.002 seconds
Max: 0.005 seconds
Mi